In [38]:
import pandas as pd
import string
from glove import Glove

In [64]:
# columns to use
cols=['description', 'points']

# import data
reviews_1 = pd.read_csv('../../data/wine-reviews/winemag-data_first150k.csv', index_col=False, usecols=cols)
reviews_2 = pd.read_csv('../../data/wine-reviews/winemag-data-130k-v2.csv', index_col=False, usecols=cols)

print("Number of entries in dataset 1: %s" %reviews_1.shape[0])
print("Number of entries in dataset 2: %s" %reviews_2.shape[0])

duplicates = set(reviews_1.description).intersection(set(reviews_2.description))

print("\nNumber of duplicate entries across datasets: %s" % len(duplicates))

# concatenate and drop duplicates
data = pd.concat([reviews_1,reviews_2])
data.drop_duplicates(inplace=True)

print("\nNumber of unique reviews: %s" % data.shape[0])

data.head()

Number of entries in dataset 1: 150930
Number of entries in dataset 2: 129971

Number of duplicate entries across datasets: 48346

Number of unique reviews: 169461


,description,points
0,This tremendous 100% varietal wine hails from ...,96
1,"Ripe aromas of fig, blackberry and cassis are ...",96
2,Mac Watson honors the memory of a wine once ma...,96
3,"This spent 20 months in 30% new French oak, an...",96
4,"This is the top wine from La Bégude, named aft...",95


In [135]:
punc_remove = string.punctuation
punc_remove = punc_remove.replace('%', '')
table = str.maketrans(dict.fromkeys(punc_remove))

# lowercase
data['description_test'] = data.description_test.str.lower()
# remove punctuation
data['description_test'] = data.description_test.str.translate(table)
# replace percentage sign
data['description_test'] = data.description_test.str.replace('%', ' percent')
# split words
data['description_test'] = data.description_test.str.split()

In [136]:
data.head()

,description,points,description_test
0,This tremendous 100% varietal wine hails from ...,96,"[this, tremendous, 100, percent, varietal, win..."
1,"Ripe aromas of fig, blackberry and cassis are ...",96,"[ripe, aromas, of, fig, blackberry, and, cassi..."
2,Mac Watson honors the memory of a wine once ma...,96,"[mac, watson, honors, the, memory, of, a, wine..."
3,"This spent 20 months in 30% new French oak, an...",96,"[this, spent, 20, months, in, 30, percent, new..."
4,"This is the top wine from La Bégude, named aft...",95,"[this, is, the, top, wine, from, la, bégude, n..."
